In [1]:
from binance import Client
import pandas as pd
from bs4 import BeautifulSoup
import requests
import time
import schedule
import math
import scipy.stats as st
import ta
import statistics
import smtplib
import os
import time
from time import sleep
import re
import pandas as pd
import matplotlib.pyplot as plt
import pandas_ta as ta
import numpy as np
import datetime

api_key="pTZlrkaCbTEoa17nor24Tw53OWYFlMBQIOmaftSLcjg5lASfhZ0048zcxOZLrBgN"
api_secret="PxcyhPku5uQon1krBa3glze2sF7oZQ3gqb9GRVkh25OFvCtyAkML139l4z9d9BwG"
client=Client(api_key,api_secret)
client.get_account()

{'makerCommission': 10,
 'takerCommission': 10,
 'buyerCommission': 0,
 'sellerCommission': 0,
 'canTrade': True,
 'canWithdraw': True,
 'canDeposit': True,
 'updateTime': 1659581801740,
 'accountType': 'SPOT',
 'balances': [{'asset': 'BTC', 'free': '0.00332900', 'locked': '0.00000000'},
  {'asset': 'LTC', 'free': '0.00059199', 'locked': '0.00000000'},
  {'asset': 'ETH', 'free': '0.02843544', 'locked': '0.00000000'},
  {'asset': 'NEO', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'BNB', 'free': '0.05222635', 'locked': '0.00000000'},
  {'asset': 'QTUM', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'EOS', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'SNT', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'BNT', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'GAS', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'BCC', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'USDT', 'free': '44.50434789', 'locked

In [2]:
def getdata(symbol, interval, lookback):
    frame=pd.DataFrame(client.get_historical_klines(symbol,interval,lookback))
    frame=frame.iloc[:,:6]
    frame.columns=['Time','Open','High','Low', 'Close','Volume']
    frame=frame.set_index('Time')
    frame.index=pd.to_datetime(frame.index,unit='ms')
    frame = frame.astype(float)
    frame['rsi']=ta.momentum.rsi(frame.Close, window=14)
    frame["EMA_20"] = ta.ema(frame.Close, length=20)
    frame["Close Lag 5"]=frame['Close'].shift(5, axis=0)
    frame["Return 5m"]=(frame['Close']-frame['Close Lag 5'])/frame['Close Lag 5']
    frame['CRYPTO']=symbol
    frame['Date_time'] = frame.index
    frame['Height']=abs(frame['High']-frame['Low'])
    frame['Vol std'] = frame['Volume'].rolling(50).std()
    return frame

def getprice(symbol, interval, lookback):
    frame=pd.DataFrame(client.get_historical_klines(symbol,interval,lookback))
    frame=frame.iloc[:,:6]
    frame.columns=['Time','Open','High','Low', 'Close','Volume']
    frame=frame.set_index('Time')
    frame.index=pd.to_datetime(frame.index,unit='ms')
    frame = frame.astype(float)
    return frame

#Checking the quantity of decimals to put in the qty of the order
def check_decimals(symbol):
    info = client.get_symbol_info(symbol)
    val = info['filters'][2]['stepSize']
    decimal = 0
    is_dec = False
    for c in val:
        if is_dec is True:
            decimal += 1
        if c == '1':
            break
        if c == '.':
            is_dec = True
    return decimal

In [3]:
#Creating the list of all the cryptos that can be exchange with USDT
exchange_info = client.get_exchange_info()
cryptos=[]
for s in exchange_info['symbols']:
    sy=s['symbol']
    if re.search('..USDT',sy):
        cryptos.append(sy)
        

crynotaval=[]
for cry in cryptos:
    try:
        df_1=getprice(cry,'1m','30 h ago UTC')
    except:
        crynotaval.append(cry)

for cry in cryptos:
    if cry in crynotaval :
        cryptos.remove(cry)


cryptos=cryptos[51:75]


# BREAKOUT STRATEGY:
For this strategy we will use 4 indicators:
1. The pvt: this should go up by 3-4 standard deviations - 1 min chart
2. the Volumen Oscilator: Change in more than 4 std- 1 min chart (For this the std should be calculated based on past information)
3. The MACD for divergence to get the momentum of the crypto.
4. The price should break the 5 min bollinger bands.

For this strategy we will see if in the last -10 min each of the cryptos has comply on at least 3 of them and buy if that is the case

## Calculating the MACD
The MACD will be use for divergence, this means to calculate the momentum of the crypto and this should be greater than 3 times the standar deviation

## Bollinger Bands
The bollinger bands are use as confirmation of the big momentum so the price should overpass the upper limit. The bollinger bands are calculated as the mean + 2std. In this case we will verify if in any of the last 5 min close prices the bollinger band was crossed

In [ ]:
breakouts=pd.DataFrame()
initial=datetime.datetime.now()
timeRun=0
while timeRun<30000:
    for cry in cryptos:
        final=datetime.datetime.now()
        timeRun=(final-initial).seconds
        qty=0
        df_f=pd.DataFrame()
        open_position=False
        boll_flag=False
        start=False
        filter1=False
        chaopapa=False
        try:
            df_1=getdata(cry,'1m','30 h ago UTC')
            price1=df_1.Close.iloc[-1]
            df_1['VEMASLOW_30']=ta.ema(df_1.Volume, length=30)
            df_1['VEMAFAST_3']=ta.ema(df_1.Volume, length=3)
            df_1['Vol_Osc_3']=df_1['VEMAFAST_3']-df_1['VEMASLOW_30']
            df_1['Vol_Flag_3_2']=np.where(df_1['Vol_Osc_3']>=2*df_1['Vol std'],"Yes","No")
            macd_df=df_1.ta.macd(close='close', fast=3, slow=26, signal=15, append=True)
            df_1['MACD']=macd_df[macd_df.columns[0]]
            df_1['MACD_H']=macd_df[macd_df.columns[1]]
            df_1['MACD_S']=macd_df[macd_df.columns[2]]
            df_1['MACD_std'] = df_1['MACD_H'].rolling(50).std()
            df_1['MACD_ratio']=df_1['MACD_H']/df_1['MACD_std']

            df_1['Price_std'] = df_1['Close'].rolling(20).std()
            df_1['Price_mean'] = df_1['Close'].rolling(20).mean()
            df_1['Boll_up'] = df_1['Price_mean']+2*df_1['Price_std']
            df_1['Boll_down'] = df_1['Price_mean']-2*df_1['Price_std']

            if sum(df_1['Close'][-12:-1]>=df_1['Boll_up'][-12:-1])<2 and sum(df_1['Close'][-12:-1]<=df_1['Boll_down'][-12:-1])<=2:
                boll_flag=True

            def boll(df_1):
                if df_1['Close']>=df_1['Boll_up']:
                    return "Yes"
                else:
                    return "No"
            df_1['Boll']=df_1.apply(boll,axis=1)
            #Height filter for the cryptos with low movements
            if sum(df_1['Height'][-8:-1]==0)<=2:
                filter1=True

            #Return filter for those with high volatility
            eight_min_ret=float(abs((df_1['Close'][-3]-df_1['Close'][-9])/df_1['Close'][-9]))
            one_min_ret=float(abs((df_1['Close'][-1]-df_1['Open'][-1])/df_1['Open'][-1]))
            
            def final_flag(df_1):
                    if df_1['Vol_Flag_3_2']=="Yes" and df_1['MACD_H']>3.3*df_1['MACD_std'] and df_1['MACD']>0 and boll_flag==True and df_1['Boll']=="Yes" and filter1==True and one_min_ret>=eight_min_ret:
                        return "Yes"
                    else:
                        return "No"

            df_1['final_flag']=df_1.apply(final_flag,axis=1)

            df_f_1=df_1[df_1['final_flag']=="Yes"]
        
            #Cheking that the chandlesticks are green
            vol1=df_1['Volume'].iloc[-1]
            vol2=df_1['Volume'].iloc[-2]
            vol3=df_1['Volume'].iloc[-3]
            chandle3val=df_1['Close'].iloc[-3]-df_1['Open'].iloc[-3]
            chandle3= 1 if chandle3val >0 else 0
            chandle2val=df_1['Close'].iloc[-2]-df_1['Open'].iloc[-2]
            chandle2= 1 if chandle2val >0 else 0
            chandle1val=df_1['Close'].iloc[-1]-df_1['Open'].iloc[-1]
            chandle1=1 if chandle1val >0 else 0
            final_val=df_1['final_flag'].iloc[-1]
            chandleRes= chandle1 + chandle3 +chandle2
        
            #Checking that the breakout just begun by confirming there the previous chandle sticks heights:
            if df_1['final_flag'].iloc[-4] == "No" or df_1['final_flag'].iloc[-3] == "No" :
                start=True
            if final_val=="Yes" and start==True:
                print("Enter to price verfication"+cry)
                price2=getprice(cry,'1m','5 m ago UTC')['Close'].iloc[-1]
                if price2>price1:
                    chaopapa=True
                else:
                    time.sleep(5)
                    price3=getprice(cry,'1m','5 m ago UTC')['Close'].iloc[-1]
                    if price3>price1:
                        chaopapa=True
                    
                if chaopapa==True:
                    price=df_1['Close'].iloc[-1]
                    dec=check_decimals(cry)
                    qty = round(20 / price, dec)
                    order=client.create_order(symbol=cry, side='BUY', type='MARKET', quantity=qty)
                    buyprice=float(order['fills'][0]['price'])
                    print('Buy Crypto' + cry + str(buyprice))
                    maxi=0.0
                    startTime=datetime.datetime.now()
                    sell=False
                    open_position=True
                    while open_position==True:
                        time.sleep(1.5)
                        dfsell= getdata(cry,'1m','10 m ago UTC')
                        price=dfsell.Close.iloc[-1]
                        gain=dfsell.Close.iloc[-1]-buyprice
                        FinalTime=datetime.datetime.now()
                        deltaTime=(FinalTime-startTime).seconds
                        if price <= buyprice*0.989 and deltaTime<400:
                            print('Enter Sell rule1: ' + str(price) + "<="+ str(buyprice * 0.95))
                            order=client.create_order(symbol=cry,side='SELL', type= 'MARKET', quantity=qty)
                            try:
                                mycursor.execute("INSERT INTO crypto.excrecords (Date, Coin, Price, AmountUSD, AmountCry, TypeExc) VALUES(NOW(), %s, %s, %s , %s,'SELL_loss')",(cry,price,amountUS,qty))
                                mydb.commit()
                            except:
                                pass
                            break
                        if price <= buyprice*0.999 and deltaTime>=400:
                            print('Enter Sell rule2: ' + str(price) + "<="+ str(buyprice * 0.999))
                            order=client.create_order(symbol=cry,side='SELL', type= 'MARKET', quantity=qty)
                            try:
                                mycursor.execute("INSERT INTO crypto.excrecords (Date, Coin, Price, AmountUSD, AmountCry, TypeExc) VALUES(NOW(), %s, %s, %s , %s,'SELL_loss')",(cry,price,amountUS,qty))
                                mydb.commit()
                            except:
                                pass
                            break

                        else:   
                            if price>(buyprice*1.0015) or sell==True:
                                sell=True
                                if maxi<=gain:
                                    maxi=gain
                                    print("new max:" + str(maxi))
                                elif gain<= maxi*0.01 and sell==True and deltaTime<120 :
                                    print('Enter Sell rule3: ' + str(gain) + "<="+ str(maxi * 0.992))
                                    order=client.create_order(symbol=cry,side='SELL', type= 'MARKET', quantity=qty)
                                    amountUS=round(price*qty,4)
                                    price=round(price,4)
                                    try:
                                        mycursor.execute("INSERT INTO crypto.excrecords (Date, Coin, Price, AmountUSD, AmountCry, TypeExc) VALUES(NOW(), %s, %s, %s , %s,'SELL_win')",(cry,price,amountUS,qty))
                                        mydb.commit()
                                    except:
                                        pass
                                    break 
                                elif gain<= maxi*0.6 and sell==True and deltaTime>120 and deltaTime<1600 :
                                    print('Enter Sell rule4: ' + str(gain) + "<="+ str(maxi * 0.9))
                                    order=client.create_order(symbol=cry,side='SELL', type= 'MARKET', quantity=qty)
                                    amountUS=round(price*qty,4)
                                    price=round(price,4)
                                    try:
                                        mycursor.execute("INSERT INTO crypto.excrecords (Date, Coin, Price, AmountUSD, AmountCry, TypeExc) VALUES(NOW(), %s, %s, %s , %s,'SELL_win')",(cry,price,amountUS,qty))
                                        mydb.commit()
                                    except:
                                        pass
                                    break

                                elif gain<= maxi*0.95 and sell==True and deltaTime>1600 :
                                    print('Enter Sell rule5: ' + str(gain) + "<="+ str(maxi * 0.95))
                                    order=client.create_order(symbol=cry,side='SELL', type= 'MARKET', quantity=qty)
                                    amountUS=round(price*qty,4)
                                    price=round(price,4)
                                    try:
                                        mycursor.execute("INSERT INTO crypto.excrecords (Date, Coin, Price, AmountUSD, AmountCry, TypeExc) VALUES(NOW(), %s, %s, %s , %s,'SELL_win')",(cry,price,amountUS,qty))
                                        mydb.commit()
                                    except:
                                        pass
                                    break
        except:
            pass
            print('Not Possible: '+ cry)
        



Enter to price verficationBANDUSDT
Buy CryptoBANDUSDT1.728
new max:0.0050000000000001155
Enter Sell rule3: -0.0020000000000000018<=0.0049600000000001145
Not Possible: KEYUSDT
Not Possible: DOCKUSDT


In [ ]:
#order=client.create_order(symbol=cry,side='SELL', type= 'MARKET', quantity=qty)